In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import time

In [2]:
def scrape_products_from_page(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find all product name elements
    product_elements = soup.find_all('div', class_='item-product')
    len(product_elements)
    products=[]
    # Extract and print the names
    for product in product_elements:
        try:
            # Product ID
            sku_div = product.find("span", class_="product-reference")
            product_id = sku_div.text.strip("[] \n") if sku_div else None

            # Product Link
            link_tag = product.find("h2", class_="product-title")
            a = link_tag.find("a")
            product_link = a['href'] if link_tag else None

            # Product Full Name
            full_name = a.text.strip() if a else None

            # Image URL
            img_tag = product.find("img", class_="img-responsive")
            image_url = img_tag['src'] if img_tag else None

            # Price
            price_tag = product.find("span", class_="price")
            price = price_tag.text.strip() if price_tag else None

            specs = {}
            #Additional data from product page
            if product_link:
                response = requests.get(product_link)
                if response.status_code == 200:
                    product_page = BeautifulSoup(response.text, 'html.parser')
                    dl = product_page.find("dl",class_="data-sheet")
                    if dl:
                        dts = dl.find_all('dt', class_='name')
                        dds = dl.find_all('dd', class_='value')

                        for dt, dd in zip(dts, dds):
                            key = dt.get_text(strip=True)
                            value = dd.get_text(strip=True)
                            specs[key] = value


            # Append product data
            product_dict = {
                "id": product_id,
                "lien": product_link,
                "nom": full_name,
                "image_url": image_url,
                "prix": price,
                "boutique":"tunisianet"
            }

            # Add specs dictionary to product_data (expand keys dynamically)
            product_dict.update(specs)

            # Append to products list
            products.append(product_dict)

        except Exception as e:
            print(f"Error processing product: {e}")
    return products

In [3]:
base_url = "https://www.tunisianet.com.tn/702-ordinateur-portable"

response = requests.get(base_url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# Find all <li> elements inside the pagination
page_items = soup.select("ul.page-list li a")

# Extract text content and convert to integers if possible
page_numbers = []
for item in page_items:
    try:
        num = int(item.text.strip())
        page_numbers.append(num)
    except ValueError:
        continue

# Get the maximum page number
last_page = max(page_numbers) if page_numbers else 1

print("Last page number:", last_page)

Last page number: 34


In [5]:
base_url = "https://www.tunisianet.com.tn/702-ordinateur-portable?page={}"
all_products = []

for page_num in tqdm(range(1, last_page+1), desc="Scraping pages"):
    #print(f"Scraping page {page_num}...")
    url = base_url.format(page_num)
    page_products = scrape_products_from_page(url)
    all_products.extend(page_products)
    time.sleep(1) #A small delay between requests to avoid hammering the server.

print(f"Total products scraped: {len(all_products)}")

Scraping pages: 100%|██████████████████████████████████████████████████████████████████| 34/34 [14:17<00:00, 25.21s/it]

Total products scraped: 812


In [6]:
all_products[0]

{'id': '82LX00EAFE-4-SAC',
 'lien': 'https://www.tunisianet.com.tn/pc-portable-tunisie/86145-pc-portable-lenovo-ideapad-1-15ijl7-celeron-n4500-4-go-256-go-ssd-windows-11-gris.html',
 'nom': 'Pc Portable Lenovo IdeaPad 1 15IJL7 / Celeron N4500 / 4 Go / 256 Go SSD / Windows 11 / Gris',
 'image_url': 'https://www.tunisianet.com.tn/430103-home/pc-portable-lenovo-ideapad-1-15ijl7-celeron-n4500-4-go-256-go-ssd-windows-11-gris.jpg',
 'prix': '689,000 DT',
 'boutique': 'tunisianet',
 "Système d'exploitation": 'Windows 11',
 'Processeur': 'Intel Celeron',
 'Réf processeur': 'Intel Celeron Dual Core N4500, 1.10 Ghz up to 2.8 Ghz , 4 Mo de mémoire cache',
 'Mémoire': '4 Go',
 'Disque Dur': '256 Go SSD',
 'Carte Graphique': 'Graphique Intégrée',
 'Réf Carte graphique': 'Intel UHD Graphics',
 'Taille Ecran': '14" - 15.6"',
 'Type Ecran': 'HD',
 'Ecran Tactile': 'Non',
 'Garantie': '1 an',
 'Couleur': 'Gris'}

In [7]:
list(all_products[0].keys())

['id',
 'lien',
 'nom',
 'image_url',
 'prix',
 'boutique',
 "Système d'exploitation",
 'Processeur',
 'Réf processeur',
 'Mémoire',
 'Disque Dur',
 'Carte Graphique',
 'Réf Carte graphique',
 'Taille Ecran',
 'Type Ecran',
 'Ecran Tactile',
 'Garantie',
 'Couleur']

In [8]:
columns_to_keep = ['id', 'lien', 'nom', 'Marque', 'image_url', 'boutique', 'Taille Ecran',
                   'Type Ecran', 'Processeur', 'Réf processeur', 'Mémoire','Disque Dur', 
                   'Type de disque dur', 'Carte Graphique', "Système d'exploitation",'Gamer', 'prix']

# Filter the dictionary to keep only the desired columns
filtered_product_features = [{key: product.get(key) for key in columns_to_keep}
                             for product in all_products]

In [9]:
df = pd.DataFrame(filtered_product_features)
df.head()

,id,lien,nom,Marque,image_url,boutique,Taille Ecran,Type Ecran,Processeur,Réf processeur,Mémoire,Disque Dur,Type de disque dur,Carte Graphique,Système d'exploitation,Gamer,prix
0,82LX00EAFE-4-SAC,https://www.tunisianet.com.tn/pc-portable-tuni...,Pc Portable Lenovo IdeaPad 1 15IJL7 / Celeron ...,None,https://www.tunisianet.com.tn/430103-home/pc-p...,tunisianet,"14"" - 15.6""",HD,Intel Celeron,"Intel Celeron Dual Core N4500, 1.10 Ghz up to ...",4 Go,256 Go SSD,None,Graphique Intégrée,Windows 11,None,"689,000 DT"
1,82LX00CFFG,https://www.tunisianet.com.tn/pc-portable-tuni...,Pc Portable Lenovo IdeaPad 1 15IJL7 / Celeron ...,None,https://www.tunisianet.com.tn/403274-home/pc-p...,tunisianet,"14"" - 15.6""",HD,Intel Celeron,"Intel Celeron Dual Core N4500, 1.10 Ghz up to ...",8 Go,256 Go SSD,None,Graphique Intégrée,FreeDos,None,"709,000 DT"
2,82LX00CKFG,https://www.tunisianet.com.tn/pc-portable-tuni...,Pc Portable Lenovo IdeaPad 1 15IJL7 / Celeron ...,None,https://www.tunisianet.com.tn/428260-home/pc-p...,tunisianet,"14"" - 15.6""",HD,Intel Celeron,"Intel Celeron Dual Core N4500, 1.10 Ghz up to ...",8 Go,256 Go SSD,None,Graphique Intégrée,FreeDos,None,"709,000 DT"
3,A30XQEA,https://www.tunisianet.com.tn/pc-portable-tuni...,Pc Portable HP 15-fd0298nk / N100 / 4 Go / 256...,None,https://www.tunisianet.com.tn/391207-home/pc-p...,tunisianet,"14"" - 15.6""",HD,Intel Celeron,"Intel Celeron N100 (jusqu’à 3,4 GHz, 6 Mo de m...",4 Go,256 Go SSD,None,Graphique Intégrée,FreeDos,None,"719,000 DT"
4,82LX00CFFG-SAC,https://www.tunisianet.com.tn/pc-portable-tuni...,Pc Portable Lenovo IdeaPad 1 15IJL7 / Celeron ...,None,https://www.tunisianet.com.tn/404184-home/pc-p...,tunisianet,"14"" - 15.6""",HD,Intel Celeron,"Intel Celeron Dual Core N4500, 1.10 Ghz up to ...",8 Go,256 Go SSD,None,Graphique Intégrée,FreeDos,None,"719,000 DT"


In [10]:
df.describe().T

,count,unique,top,freq
id,812,812,82LX00EAFE-4-SAC,1
lien,812,812,https://www.tunisianet.com.tn/pc-portable-tuni...,1
nom,812,795,Pc portable Dell Latitude 5550 / Ultra 7 155U ...,3
Marque,0,0,NaN,NaN
image_url,812,812,https://www.tunisianet.com.tn/430103-home/pc-p...,1
boutique,812,1,tunisianet,812
Taille Ecran,789,7,"14"" - 15.6""",571
Type Ecran,745,7,Full HD,583
Processeur,806,17,Intel Core i5 | Intel Core 5,255
Réf processeur,800,74,"Intel Core i7-13620H (4.9 GH, 24 Mo de mémoire...",107


In [12]:
# Save to CSV
df.to_csv("scraped_data/tunisianet_products.csv", index=False, encoding='utf-8-sig')

print("Data saved to tunisianet_products.csv")

Data saved to tunisianet_products.csv
